In [1]:
print("Hello|")

Hello|


In [3]:
pip install -U langchain-community


  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached marshmallow-3.22.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 357.1 kB/s eta 0:00:06
   -------- ------------------------------- 0.5/2.4 MB 357.1 kB/s eta 0:00:06
  

In [4]:
pip install -U langchain pinecone-client ctransformers


  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.9 MB 6.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/9.9 MB 6.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/9.9 MB 6.1 MB/s eta 0:00:02
   ------------------- -------------------- 4.7/9.9 MB 6.1 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.9 MB 5.8 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/9.9 MB 5.7 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/9.9 MB 5.7 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.9 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.9 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 5.0 MB/s eta 0:00:00


In [7]:
from tqdm.autonotebook import tqdm

In [8]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers

In [ ]:
PINECONE_API_KEY = "ddbe248c-a58c-49ea-8929-26097433ff34"

In [15]:
#Extract data from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(data,
                             glob="D:/Code/Medical chatbot/Medical-Chatbot-using-llama/Data/Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf",
                             loader_cls=PyMuPDFLoader)
    documents = loader.load()

    return documents



NameError: name 'documents' is not defined